In [1]:
#Prediction using Tensorflow LibrarY

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense
from dateutil.parser import parse

In [2]:
#Read csv file 
df=pd.read_csv('dataResearch.csv')
df


,SL.,Station,Year,Month,Date,Temperature (maximum),Temperature(minimum),Temperature(Average),Rainfall(mm),Humidity(percentage)
0,1,Sylhet (10705),1970,1,1/1/1970,23.90,11.37,17.635,62,76.52
1,2,Sylhet (10705),1970,2,1/2/1970,26.85,14.07,20.460,65,67.96
2,3,Sylhet (10705),1970,3,1/3/1970,29.92,17.86,23.890,60,66.26
3,4,Sylhet (10705),1970,4,1/4/1970,30.96,21.65,26.305,466,76.10
4,5,Sylhet (10705),1970,5,1/5/1970,31.22,23.08,27.150,851,80.84
...,...,...,...,...,...,...,...,...,...,...
631,632,Sylhet (10705),2022,8,1/8/2022,34.30,26.20,30.250,461,80.58
632,633,Sylhet (10705),2022,9,1/9/2022,33.28,25.57,29.425,744,84.63
633,634,Sylhet (10705),2022,10,1/10/2022,32.11,23.47,27.790,334,82.29
634,635,Sylhet (10705),2022,11,1/11/2022,31.32,19.55,25.435,0,73.53


In [3]:
#Checking null values
df.isnull().sum()


SL.                      0
Station                  0
Year                     0
Month                    0
Date                     0
Temperature (maximum)    0
Temperature(minimum)     0
Temperature(Average)     0
Rainfall(mm)             0
Humidity(percentage)     0
dtype: int64

In [4]:
total_null_values = df.isnull().sum().sum()

print(f"Total null values in the DataFrame: {total_null_values}")

Total null values in the DataFrame: 0


In [5]:
#Removing first 2 columns

data = pd.read_csv('dataResearch.csv', usecols=lambda column: column not in ['SL.', 'Station'])

In [6]:
data

,Year,Month,Date,Temperature (maximum),Temperature(minimum),Temperature(Average),Rainfall(mm),Humidity(percentage)
0,1970,1,1/1/1970,23.90,11.37,17.635,62,76.52
1,1970,2,1/2/1970,26.85,14.07,20.460,65,67.96
2,1970,3,1/3/1970,29.92,17.86,23.890,60,66.26
3,1970,4,1/4/1970,30.96,21.65,26.305,466,76.10
4,1970,5,1/5/1970,31.22,23.08,27.150,851,80.84
...,...,...,...,...,...,...,...,...
631,2022,8,1/8/2022,34.30,26.20,30.250,461,80.58
632,2022,9,1/9/2022,33.28,25.57,29.425,744,84.63
633,2022,10,1/10/2022,32.11,23.47,27.790,334,82.29
634,2022,11,1/11/2022,31.32,19.55,25.435,0,73.53


In [7]:
# Extract and scale the temperature values
temperature_values = df['Temperature (maximum)'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_temperature = scaler.fit_transform(temperature_values)

In [8]:
# Create sequences for training the RNN
sequence_length = 12
sequences = []
next_temperature = []

for i in range(len(scaled_temperature) - sequence_length):
    seq = scaled_temperature[i:i + sequence_length]
    label = scaled_temperature[i + sequence_length:i + sequence_length + 1]
    sequences.append(seq)
    next_temperature.append(label)

X = np.array(sequences)
y = np.array(next_temperature)

# Split the data into training and testing sets (up to 2021)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [9]:
# Build the RNN model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [10]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

Epoch 1/50
16/16 [==============================] - 3s 5ms/step - loss: 0.3108
Epoch 2/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0907
Epoch 3/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0468
Epoch 4/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0426
Epoch 5/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0419
Epoch 6/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0411
Epoch 7/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0408
Epoch 8/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0397
Epoch 9/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0385
Epoch 10/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0381
Epoch 11/50
16/16 [==============================] - 0s 5ms/step - loss: 0.0357
Epoch 12/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0332
Epoch 13/50
16/16 [==============================

In [11]:
# Make predictions on the test set
y_pred = model.predict(X_test)


4/4 [==============================] - 0s 5ms/step


In [12]:
# Inverse transform the scaled predictions and actual values
y_pred_actual = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))


In [13]:
# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test_actual, y_pred_actual)
print(f'Mean Absolute Error on Test Set: {mae}')


NameError: name 'mean_absolute_error' is not defined

In [ ]:
# Print the forecasted temperatures
print("Forecasted Temperatures:")
for date, temp in zip(future_dates, prediction_data['Predicted_Temperature']):
    print(f"{date.strftime('%m/%Y')}: {temp:.2f}°C")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Assuming 'Date' is in datetime format
data['Date'] = pd.to_datetime(data['Date'])
future_dates = pd.date_range(start=data['Date'].max() + pd.DateOffset(months=1), periods=24, freq='M')

# Set the figure size
plt.figure(figsize=(12, 6))

# Plot actual temperature values
plt.plot(data['Date'], temperature_values, label='Actual Temperature', color='#4682B4')

# Plot predicted temperature values for the next 24 months
plt.plot(future_dates, predicted_temperature[-24:], label='Predicted Temperature', linestyle='dashed', color='red')

plt.title('Temperature Prediction using RNN')
plt.xlabel('Date')
plt.ylabel('Temperature (maximum)')
plt.legend()
plt.show()


In [1]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Assuming y_test_actual and y_pred_actual are your actual and predicted values

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test_actual, y_pred_actual)
print(f'Mean Absolute Error (MAE): {mae:.2f}°C')

# Mean Squared Error (MSE)
mse = mean_squared_error(y_test_actual, y_pred_actual)
print(f'Mean Squared Error (MSE): {mse:.2f}°C²')

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}°C')

# R-squared (R2) Score
r2 = r2_score(y_test_actual, y_pred_actual)
print(f'R-squared (R2) Score: {r2:.2f}')


NameError: name 'y_test_actual' is not defined